In [6]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

#import the Cross sections file 
openmc.config['cross_sections'] = "/home/f_z/endfb-vii.1-hdf5/cross_sections.xml"

In [7]:
#Materials definition

###Fuel Pellets###
###Uranium###
uo2=openmc.Material(material_id=1)
uo2.add_element('U',1,enrichment=5.0) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2.add_element('O',2)
uo2.set_density('g/cm3',10.3)


###Coolant###

water=openmc.Material(material_id=2)
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3',0.650)

###Assembly###

helium=openmc.Material(material_id=3)
helium.add_element('He',1)
helium.set_density('g/cm3',0.0043)


###Cladding### (from Input Correlations for Irradiation Creep of FeCrAl and SiC Based...)
#and Analysis of Options andExperimental Examination of Fuels for Water Cooled Reactors with Increased Accident Tolerance (ACTOF)

fecral=openmc.Material(material_id=4)
fecral.add_element('Fe',0.7868,'wo')
fecral.add_element('Cr',0.1302,'wo')
fecral.add_element('Al',0.0508,'wo')
fecral.add_element('Mo',0.0199,'wo')
fecral.add_element('Si',0.0021,'wo')
fecral.add_element('Nb',0.0097,'wo')
fecral.add_element('Y',0.00032,'wo')
fecral.add_element('C',0.00003,'wo')
fecral.add_element('N',0.000013,'wo')
fecral.add_element('O',0.000028,'wo')
fecral.add_element('S',0.000003,'wo')
fecral.set_density('g/cm3',7.10)
#print(fecral)

#plugs
steel= openmc.Material(material_id=5)
steel.add_element('C',0.0005,'wo')
steel.add_element('Ti',0.005,'wo')
steel.add_element('Cr',0.1625,'wo')
steel.add_element('Mn',0.02,'wo')
steel.add_element('Fe',0.65,'wo')
steel.add_element('Ni',0.14,'wo')
steel.add_element('Mo',0.022,'wo')
steel.set_density('g/cm3',7.55)

#Export the materials
materials=openmc.Materials([uo2,helium,water,fecral,steel])
materials.export_to_xml('materials.xml')

/home/f_z/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/f_z/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/f_z/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/f_z/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=4.
  warn(msg, IDWarning)
/home/f_z/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=5.
  warn(msg, IDWarning)


In [11]:
###Surfaces definition###

#lateral surface, infinite cylinder centered in z axis, outer surfaces
frod_height=385.866  
rod_pitch=1.26

r_p=0.857/2 #pellet
r_g=r_p+0.016 #gap
r_c=r_g+0.0385 #clad

s_pellet=openmc.ZCylinder(0,0,r_p)
s_gap=openmc.ZCylinder(0,0,r_g)
s_clad=openmc.ZCylinder(0,0,r_c)

#top and bottom surface
#-rodheight/2(b_bottom_plug)#bottom_plug_pos#pellet_stack#bottom_plenum_pos#top_plenum_pos(t_bottom_plug)#rodheight/2(t_top_plug)#

s_top=openmc.ZPlane(frod_height/2., boundary_type='reflective') #reflective measn that we don't have leakages
s_bottom=openmc.ZPlane(-frod_height/2, boundary_type='reflective') 

###full rod geometry, plenum only on the upper part of the rod
bottom_plug_pos=-frod_height/2+1.465
bottom_plenum_pos=frod_height/2-16.457-1.468
top_plenum_pos=frod_height/2-1.468

s_bottom_plenum= openmc.ZPlane(bottom_plenum_pos) 
s_top_plenum= openmc.ZPlane(top_plenum_pos)
s_b_top_end_plug= openmc.ZPlane(bottom_plug_pos) #surface between bottom plug and fuel stack

#let's create a geometry to run the code:
#edge = rod_pitch/2 * 1/np.sin(np.pi/3)
#hex_cell = openmc.model.SPrism(edge_length=edge,orientation='x')#,boundary_type='reflective')
square_cell = openmc.model.RectangularPrism(rod_pitch,rod_pitch)#,boundary_type='reflective')
#the commented part to be added to run only this part of the code, otherwise it will export also the reflective boundary, we don't wnat that.


In [12]:
###Cell construction ###

c_pellet=openmc.Cell(name="Pellet")
c_pellet.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet 
c_pellet.fill=uo2  

c_gap=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap.fill=helium

# gas plenum
c_gas= openmc.Cell(name='gas plenum')
c_gas.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas.fill= helium

c_clad=openmc.Cell(name="Clad")
c_clad.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad.fill=fecral

### Coolant cell ###
c_coolant=openmc.Cell(name="Coolant")
c_coolant.region= +s_bottom & -s_top & +s_clad & -square_cell# define a hexagonal region for the coolant
c_coolant.fill=water

#bottom plug
c_b_plug= openmc.Cell(name='Bottom plug')
c_b_plug.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug.fill= steel

#top plug
c_t_plug= openmc.Cell(name='top plug')
c_t_plug.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug.fill= steel

fuel_rod_universe=openmc.Universe(cells=[c_pellet,c_gap,c_gas,c_clad,c_coolant,c_t_plug,c_b_plug])


In [ ]:
'''
fuel_rod_universe.plot(basis="xz",width=(1.5,390.9),color_by= 'material') #infinitely defined lateraly
fuel_rod_universe.plot(basis="xy",width=(1.5,1.5),color_by= 'material')
fuel_rod_universe.plot(basis="yz",width=(1.5,1.5),color_by= 'material') 
'''

In [13]:
### export the geometry in a xml file

fuel_cell_geom=openmc.Geometry(fuel_rod_universe) #before we didn't define the class
fuel_cell_geom.export_to_xml('geometry.xml')


In [ ]:
#rod universe
'''
inclad_universe= openmc.Universe(cells=[c_pellet,c_gas,c_gap,c_clad,c_coolant,
                                       c_b_plug,c_t_plug])
inclad_universe.plot(basis='xz', width=(1.,385.15), color_by='material')
#
inclad= openmc.Cell(name='Components inside the cladding')
inclad.region= - s_t_top_end_plug & +s_b_bottom_end_plug &-s_gap
inclad.fill= inclad_universe

#clad, already defined c_clad 

#coolant, already defined c_coolant

fuel_rod_universe=openmc.Universe(cells=[inclad,c_clad,c_coolant]) 
'''

In [ ]:
###Tallies###

###Filters###
#for our purpose we need to use an energy filter

e_filter=openmc.EnergyFilter([.0,.68,20e6]) #2 energy groups, but for a fast reactor there are not so muche thermal ns

#meshes, where the neutrons are flying?
msh_filter=openmc.RegularMesh()#mesh_id=1) let's construct the grid, with number of nodes(dimensions)
msh_filter.dimension=[25,25,2] #x,y,z meshes generated in the different directions.this is a box,not an hexagon
msh_filter.lower_left=[-rod_pitch/2,-rod_pitch/2,-frod_height/2] #vertexes edges of the box
msh_filter.upper_right=[rod_pitch/2,rod_pitch/2,frod_height/2]
mesh_filter=openmc.MeshFilter(msh_filter)#filter_id=2)

###Scores###

#fluxes
flux=openmc.Tally(name="Flux")
flux.scores=["flux"]
flux.filters=[e_filter,mesh_filter]


#fission reaction rate 
fission_rr=openmc.Tally(name="fission_rate")
fission_rr.scores=["fission"]
flux.filters=[e_filter,mesh_filter]

In [ ]:
###exporting the tallies in a xml file

tallies=openmc.Tallies([flux,fission_rr])
tallies.export_to_xml()


In [ ]:
###run the simulation###

#Sources definitiion
source_point= openmc.stats.Point(xyz=(0.,0.,0.)) #where it starts the gen, or we can use a region
source_region=openmc.stats.Box((-0.3075,-0.3075,-0.5),(0.3075,0.3075,0.5)) #radius of the cladding, edges of the box
source_energy=openmc.stats.Watt()#by default the constrains define the rejection: 
                                 # fissionable means that it search onlyfor fissionable material
source=openmc.IndependentSource(space=source_region,energy=source_energy,constraints={"fissionable":True})
source_2=openmc.IndependentSource(space=source_point,energy=source_energy)

#simulation settings

sim_sett=openmc.Settings()
sim_sett.run_mode="eigenvalue"
sim_sett.particles=10000  #produced by the source, every time the batches rinormalize respect this number
                         #same number of neutrons per generation, every neutrons has a weight, 
                         # that means that not all the ns contribute to the
                         # in this example we are focusing on a NON criticality problem=> prim o poi i neutroni finiscono.
sim_sett.inactive=0
sim_sett.batches=10
sim_sett.source=source_2

sim_sett.export_to_xml()


In [ ]:
'''
for cell in fuel_cell_geom.get_all_cells().values():
    print(f"Cell ID: {cell.id}, Region: {cell.region}")
#the output provide a dictionary with the list of different cell that are used to create the 
#universe (in this case fuel_rod_universe). The numbers inside parenthesis rappresents the surfaces
#that composed each cell, with negative and positive def based on what you did in the definition 
#of each region

# Get all surfaces from your geometry
surfaces = fuel_cell_geom.get_all_surfaces()

# Retrieve Surface 8
surface_8 = surfaces[8]  # Assuming 8 is a valid surface ID
surface_7=surfaces[7]
# Print surface properties (to understand what it is)
print(surface_8,surface_7)
'''
import os
required_files = ["materials.xml", "geometry.xml", "settings.xml"]
for file in required_files:
    if not os.path.exists(file):
        raise RuntimeError(f"Missing required file: {file}")
    else:
        print("All the files are here")
openmc.run()

In [ ]:
###How to verify that the sources are working, use of the h5 file
'''
sp_file=openmc.StatePoint("statepoint.5.h5") #to be changed every time you re-run the code

plt.quiver(sp_file.source["r"]["x"],sp_file.source["r"]["y"],sp_file.source["u"]["x"],sp_file.source["u"]["y"],
           np.log(sp_file.source["E"]),cmap="jet",scale=15.)#help to visualize the population and directions from the source 

plt.colorbar()
plt.xlim(-1.,1.)
plt.ylim(-1.,1.)

#K_eff plot
plt.plot(sp_file.k_generation)

###tally extraction:sp_file.get_tally(scores=["fission"],name=fission_rr) you need the name of the tally
#divide for each energy groups: use the energy filter "get_slicer function", use print(.shape to see the dimensions of the arrays)
#reshape the array of the sliced array(the square of the original mesh) using the function "mean.shape =(25,25)"
fr_tally = sp_file.get_tally(scores=['fission'], name='fission_rate') #instead you can do the same for the flux
print(fr_tally.shape)
fr1 = fr_tally.get_slice(filters=[openmc.EnergyFilter], filter_bins=[((0, 0.68),)])
fr2 = fr_tally.get_slice(filters=[openmc.EnergyFilter], filter_bins=[((0.68, 20e6),)])
fr1.mean.shape = (25, 25)
fr2.mean.shape = (25, 25)

plt.rcParams['figure.figsize'] = [14, 10]
fig = plt.subplot(121)
fig1 = plt.imshow(fr1.mean, label="E < 1 keV")
fig2 = plt.subplot(122)
fig2.imshow(fr2.mean, label="E > 800 keV")
'''